In [113]:
import pandas as pd
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
import numpy as np
from dash import dash_table


In [115]:
data_death = pd.read_csv(r'C:\Users\Utilisateur\Downloads\time_series_covid19_deaths_global.csv')
data_confirmed = pd.read_csv(r'C:\Users\Utilisateur\Downloads\time_series_covid19_confirmed_global.csv')

In [117]:
#data feltration
data_confirmed['Province/State'].unique()
data_confirmed

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,703228,703228,703228,703228,703228,703228,703228,703228,703228,703228
285,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
286,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11945,11945,11945,11945,11945,11945,11945,11945,11945,11945
287,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,343012,343012,343079,343079,343079,343135,343135,343135,343135,343135


In [119]:
data_death_filtered = data_death[['Country/Region' , '3/9/23','Province/State']]
data_confirmed_filtered = data_confirmed[['Country/Region' , '3/9/23','Province/State']]
data_death_filtered

,Country/Region,3/9/23,Province/State
0,Afghanistan,7896,NaN
1,Albania,3598,NaN
2,Algeria,6881,NaN
3,Andorra,165,NaN
4,Angola,1933,NaN
...,...,...,...
284,West Bank and Gaza,5708,NaN
285,Winter Olympics 2022,0,NaN
286,Yemen,2159,NaN
287,Zambia,4057,NaN


In [121]:
data_death_filtered = data_death_filtered[data_death_filtered['Province/State'].isnull()]
data_confirmed_filtered = data_confirmed_filtered[data_confirmed_filtered['Province/State'].isnull()]


In [123]:
data_death_filtered = data_death_filtered.rename(columns={"3/9/23": "Deaths",'Country/Region' : 'Country'})
data_confirmed_filtered = data_confirmed_filtered.reindex(columns={"3/9/23": "Confirmed",'Country/Region' : 'Country'})
data_confirmed_filtered = data_confirmed_filtered.rename(columns={"3/9/23": "Confirmed",'Country/Region' : 'Country'})


In [125]:
data_death_filtered

,Country,Deaths,Province/State
0,Afghanistan,7896,NaN
1,Albania,3598,NaN
2,Algeria,6881,NaN
3,Andorra,165,NaN
4,Angola,1933,NaN
...,...,...,...
284,West Bank and Gaza,5708,NaN
285,Winter Olympics 2022,0,NaN
286,Yemen,2159,NaN
287,Zambia,4057,NaN


In [127]:
data_confirmed_filtered 

,Confirmed,Country
0,209451,Afghanistan
1,334457,Albania
2,271496,Algeria
3,47890,Andorra
4,105288,Angola
...,...,...
284,703228,West Bank and Gaza
285,535,Winter Olympics 2022
286,11945,Yemen
287,343135,Zambia


In [129]:
df = pd.merge(data_confirmed_filtered, data_death_filtered, on="Country", how="outer")

In [131]:
df['Country'] = df['Country'].replace('US', 'United States')
df = df[['Country','Confirmed','Deaths']]
df

,Country,Confirmed,Deaths
0,Afghanistan,209451,7896
1,Albania,334457,3598
2,Algeria,271496,6881
3,Andorra,47890,165
4,Angola,105288,1933
...,...,...,...
193,West Bank and Gaza,703228,5708
194,Winter Olympics 2022,535,0
195,Yemen,11945,2159
196,Zambia,343135,4057


In [133]:
population = pd.read_csv(r'C:\Users\Utilisateur\Downloads\world_population.csv')
population = population.rename(columns={"Country/Territory": "Country"})
population['Country'] = population['Country'].str.upper()
df['Country'] = df['Country'].str.upper()
df_filtrer = pd.merge(df, population, on="Country")
df_filtrer = df_filtrer.rename(columns={"2022 Population": "Population"})
df_filtrer = df_filtrer[['Country','Confirmed','Deaths','Population']]
df_filtrer['Proportion (%)'] = df_filtrer['Deaths'] * 100  / df_filtrer['Population']
df_filtrer

,Country,Confirmed,Deaths,Population,Proportion (%)
0,AFGHANISTAN,209451,7896,41128771,0.019198
1,ALBANIA,334457,3598,2842321,0.126587
2,ALGERIA,271496,6881,44903225,0.015324
3,ANDORRA,47890,165,79824,0.206705
4,ANGOLA,105288,1933,35588987,0.005431
...,...,...,...,...,...
176,VENEZUELA,552162,5854,28301696,0.020684
177,VIETNAM,11526994,43186,98186856,0.043983
178,YEMEN,11945,2159,33696614,0.006407
179,ZAMBIA,343135,4057,20017675,0.020267


In [135]:
#filtrer les que pays europeeienes

european_countries = [
    "Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", 
    "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", 
    "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", 
    "France", "Georgia", "Germany", "Greece", "Hungary", 
    "Iceland", "Ireland", "Italy", "Kazakhstan", "Kosovo", 
    "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Malta", 
    "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", 
    "Norway", "Poland", "Portugal", "Romania", "Russia", 
    "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Turkey", "Ukraine", "United Kingdom", 
    "Vatican City"
]
df_europe_death = data_death[data_death['Country/Region'].isin(european_countries)]
#selectionner les date columns 
date_columns = df_europe_death.columns[4:]
deaths_by_date = df_europe_death[date_columns].sum()
deaths_by_date
# Transformer la Series en DataFrame
deaths_df = deaths_by_date.reset_index()
# Renommer les colonnes
deaths_df.columns = ["Date", "Deaths"]
deaths_df

,Date,Deaths
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0
3,1/25/20,0
4,1/26/20,0
...,...,...
1138,3/5/23,2151455
1139,3/6/23,2151612
1140,3/7/23,2152050
1141,3/8/23,2152486


In [137]:
df_europe_confirmed = data_confirmed[data_confirmed['Country/Region'].isin(european_countries)]
#selectionner les date columns 
date_columns = df_europe_confirmed.columns[4:]
confirmed_by_date = df_europe_confirmed[date_columns].sum()
# Transformer la Series en DataFrame
confirmed_df = confirmed_by_date.reset_index()
# Renommer les colonnes
confirmed_df.columns = ["Date", "Confirmed"]
confirmed_df

,Date,Confirmed
0,1/22/20,0
1,1/23/20,0
2,1/24/20,2
3,1/25/20,3
4,1/26/20,3
...,...,...
1138,3/5/23,265705172
1139,3/6/23,265735265
1140,3/7/23,265791685
1141,3/8/23,265853664


In [139]:
# combiner les deux base de données
df_date = pd.merge(confirmed_df, deaths_df, on="Date")
df_date

,Date,Confirmed,Deaths
0,1/22/20,0,0
1,1/23/20,0,0
2,1/24/20,2,0
3,1/25/20,3,0
4,1/26/20,3,0
...,...,...,...
1138,3/5/23,265705172,2151455
1139,3/6/23,265735265,2151612
1140,3/7/23,265791685,2152050
1141,3/8/23,265853664,2152486


In [141]:
# mettre tous les pays en majuscule 
pays_capitalises = [pays.upper() for pays in european_countries]
# filtrer les pays européens 
df_table = df[df['Country'].isin(pays_capitalises)]
df_table['Deaths/Confirmed (%)'] = df_table['Deaths'] * 100 / df_table['Confirmed']
df_table = df_table.sort_values(by='Deaths/Confirmed (%)' ,ascending= False)  # trier  par rapport Deaths/Confirmed
df_table = df_table.reset_index(drop = True) # refaire l'index des pays pour qu'il soit ordonné
top_10 = df_table.tail(10)
top_10 = top_10.sort_values(by = 'Deaths/Confirmed (%)' )
last_10 = df_table.head(10)
last_10
df_table

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_4416\1044673874.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Confirmed,Deaths,Deaths/Confirmed (%)
0,BOSNIA AND HERZEGOVINA,401729,16280,4.052483
1,BULGARIA,1297523,38228,2.946229
2,NORTH MACEDONIA,346852,9662,2.785626
3,HUNGARY,2196804,48762,2.219679
4,UKRAINE,5711929,119283,2.088314
5,ROMANIA,3346046,67736,2.024359
6,MOLDOVA,611140,12003,1.964034
7,ARMENIA,447308,8727,1.951005
8,POLAND,6444960,119010,1.846559
9,RUSSIA,22075858,388478,1.759741


In [143]:
df_filtrer

,Country,Confirmed,Deaths,Population,Proportion (%)
0,AFGHANISTAN,209451,7896,41128771,0.019198
1,ALBANIA,334457,3598,2842321,0.126587
2,ALGERIA,271496,6881,44903225,0.015324
3,ANDORRA,47890,165,79824,0.206705
4,ANGOLA,105288,1933,35588987,0.005431
...,...,...,...,...,...
176,VENEZUELA,552162,5854,28301696,0.020684
177,VIETNAM,11526994,43186,98186856,0.043983
178,YEMEN,11945,2159,33696614,0.006407
179,ZAMBIA,343135,4057,20017675,0.020267


In [148]:
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Charger les données COVID-19 (remplace par ton propre jeu de données)
# Exemple de DataFrame, tu devras l'ajuster en fonction de tes données

# Initialiser l'application Dash
app = dash.Dash(__name__)

custom_colorscale = ["#08da55", "#f9d107", "#fe0e0e"]  # Bleu, Jaune, Rouge

# Définir la disposition de l'application avec Dropdown et Graph , table , 
app.layout = html.Div(
    children=[
        html.H1("COVID-19: Cas confirmés et nombre de décès par pays", style={"textAlign": "center"}),
        dcc.Dropdown(
            id="dropdown-key",
            options=[
                {"label": "Cas confirmés", "value": "Confirmed"},
                {"label": "Décès", "value": "Deaths"},
                {"label": "Proportion des Décès par rapport à la population", "value": "Proportion (%)"},
                {"label": "Population", "value": "Population"}
            ],
            value="Confirmed",  # Valeur par défaut
            style={"width": "70%", "margin": "0 auto", "marginBottom": "20px"}
        ),
        dcc.Graph(id="choropleth-map"),  # Carte choroplèthe
        html.H1("Graphe des cas confirmés et décès en Europe par rapport au temp", style={"textAlign": "center"}),
        dcc.Dropdown(
            id='drop_graphe',
            options=[
                {'label': 'Cas confirmés', 'value': 'Confirmed'},
                {'label': 'Décès', 'value': 'Deaths'}
            ],
            value="Confirmed",  # Valeur par défaut
            style={"width": "70%", "margin": "0 auto", "marginBottom": "20px"}
        ),
        dcc.Graph(id='xy-graph'),  # Graphique des cas confirmés et décès
        html.H1("Classement des pays européens du nombre de décès sur total des cas confirmés: 10 premiers, 10 derniers", style={"textAlign": "center"}),

        # Dropdown pour basculer entre Top 10 et Last 10
        dcc.Dropdown(
            id="dropdown-table",
            options=[
                {"label": "Top 10", "value": "top"},
                {"label": "Last 10", "value": "last"}
            ],
            value="top",  # Valeur par défaut
            style={"width": "50%", "margin": "0 auto", "marginBottom": "20px"}
        ),

        # Tableau interactif
        dash_table.DataTable(
            id="data-table",
            columns=[
                {"name": col, "id": col} for col in df_table.columns
            ],
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "center", "padding": "10px"},
            style_header={"backgroundColor": "lightblue", "fontWeight": "bold"},
        )
    ],
    style={
        "display": "flex",
        "flexDirection": "column",
        "alignItems": "center",
        "justifyContent": "flex-start",
        "height": "100vh"
    }
)

# Callback pour mettre à jour la carte choroplèthe
@app.callback(
    Output("choropleth-map", "figure"),
    Input("dropdown-key", "value")
)

def update_map(selected_key):
    fig = px.choropleth(
        df_filtrer,
        locations="Country",
        locationmode="country names",
        color=selected_key,
        hover_name="Country",
        hover_data={selected_key: True},
        title=f"Carte basée sur {selected_key}",
        color_continuous_scale=custom_colorscale
    )
    
    fig.update_layout(
        margin={"r": 0, "t": 50, "l": 0, "b": 0},
        height=600,
        width=1000
    )
    return fig

# Callback pour mettre à jour le graphique en fonction du dropdown
@app.callback(
    Output('xy-graph', 'figure'),
    Input('drop_graphe', 'value')
)
def update_graph(selected_key):
    fig = px.bar(
        df_date,
        x="Date",
        y=selected_key,
        title=f"{selected_key} par pays",
        labels={selected_key: selected_key, "Country": "Pays"},
        color=selected_key
        
    )
    fig.update_layout(
        height=600,
        width=1000,
        title=f"Graphe of {selected_key} In Europe based in time"
    )
    return fig
@app.callback(
    Output("data-table", "data"),
    Input("dropdown-table", "value")
)
def update_table(selected_option):
    if selected_option == "top":
        return top_10.to_dict("records")
    elif selected_option == "last":
        return last_10.to_dict("records")

# Lancer l'application
if __name__ == "__main__":
    app.run_server(debug=True, port=8083)
